Suppose we want to build a data set for a Computer vision task that involves gender images. 
Your tasks are the following:
Collect 10k male/female images from:
https://www.imdb.com
Make sure to render the whole page using selenium and then use BeautifulSoup  to scrape the images
Create a folder for male/female
Each image will be named after the person in the picture


In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
import requests
import os
import threading

driver = webdriver.Chrome(ChromeDriverManager().install())

image_data = []  # Store the image data (name and URL)

for page in range(1, 101):
    url = f'https://www.imdb.com/search/name/?gender=male&has=awards&count=100&start={page * 100}&ref_=rlm'
    #url = f'https://www.imdb.com/search/name/?gender=female&has=awards&count=100&start={page * 100}&ref_=rlm'
    driver.get(url)
    time.sleep(5)

    try:
        for person in range(1, 101):
            name_element = driver.find_element('xpath', f'(//div[@class="lister-item-content"]/h3/a)[{person}]')
            name = name_element.text
            img_element = driver.find_element('xpath', f'(//div[@class="lister-item-image"]/a/img)[{person}]')
            img_src = img_element.get_attribute('src')
            image_data.append((name, img_src))

        def download_image(count, name, url, save_path):
            max_retries = 3
            retry_delay = 3  # seconds

            for retry in range(1, max_retries + 1):
                try:
                    response = requests.get(url)
                    if response.status_code == 200:
                        with open(save_path, 'wb') as f:
                            f.write(response.content)
                        print(f'Image {count} saved: {save_path}')
                        return
                    else:
                        print(f'Failed to download Image {count}: {save_path}')
                        return
                except requests.RequestException as e:
                    print(f'Error downloading Image {count} (Attempt {retry}/{max_retries}): {str(e)}')
                    time.sleep(retry_delay)

            print(f'Failed to download Image {count}: {save_path}')

        # Create a list to store threads
        threads = []
        for i, (name, url) in enumerate(image_data, start=1):
            file_name = f'{name}.jpg'
            save_path = os.path.join('C:\\Users\\MD Arifuzzaman\\Documents\\Integrify_Assignment\\images', file_name)
            thread = threading.Thread(target=download_image, args=(i, name, url, save_path))
            thread.start()
            threads.append(thread)

        # Wait for all threads to complete
        for thread in threads:
            thread.join()

    except Exception as e:
        print(f'An error occurred: {str(e)}')

# Close the driver
driver.quit()
